In [1]:
import csv
import json
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torchtext
from torchtext.legacy.data import Field, LabelField, TabularDataset, BucketIterator

import transformers
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert = BertModel.from_pretrained('bert-base-uncased')

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Current device:', torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Current device: NVIDIA GeForce RTX 3070 Laptop GPU


In [2]:
# tokeniser test
tokenizer.encode('Yo this is Daniel!')

[101, 10930, 2023, 2003, 3817, 999, 102]

In [3]:
# load dataset: transfer to json type so that torchtext reads it
def prepare_train_dataset():
        
    # X = [[sentence1, word1], [sentence2, word2], ...], y = [score1, score2, ...]
    X_tr = []
    y_tr = []
    f = open("./datasets/Sub-task 1/lcp_single_train.tsv")
    read_tsv = csv.reader(f, delimiter="\t")
    next(read_tsv, None) # skip header
    
    train_json = open('./temp/train.json', 'w')
    
    for data in read_tsv:
        d = {'sentence': data[2], 'label': float(data[4])}
        json.dump(d, train_json)
        train_json.write('\n')
    f.close()

prepare_train_dataset()

In [8]:
# text and label preprocessing - TO FIX
batch_size = 32


sos_idx = tokenizer.cls_token_id
eos_idx = tokenizer.sep_token_id
pad_idx = tokenizer.pad_token_id
unk_idx = tokenizer.unk_token_id

print(f'Special token ids: \n SOS:{sos_idx}, EOS: {eos_idx}, PAD: {pad_idx}, UNK: {unk_idx}')

# define fields
text_field = Field(use_vocab=True,
                   tokenize=tokenizer.encode,
#                    preprocessing = tokenizer.convert_tokens_to_ids,
                   batch_first=True,
                   pad_token=pad_idx,
                   unk_token=unk_idx,
                   init_token=sos_idx,
                   eos_token=eos_idx)

label_field = LabelField(use_vocab=False,
                    batch_first=True,
                    dtype=torch.float)

fields = {'sentence': ('sentence', text_field), 'label': ('label',label_field)}

# dataloader using json format
json_path = "./temp/"
train_data = TabularDataset(
    path =  json_path + 'train.json',
    format='json',
    fields = fields)

text_field.build_vocab(train_data)

# iterators
train_it = BucketIterator(train_data,
                          batch_size=batch_size,
                          sort_key= lambda x: len(x.sentence),
                          repeat=True,
                          sort=False,
                          shuffle=True,
                          device=device)
# valid_it ...
# test_it ...

print('Training terator created with', len(train_it), 'batches.')

Special token ids: 
 SOS:101, EOS: 102, PAD: 0, UNK: 100
Training terator created with 226 batches.


In [61]:
# print an example
print('########Dataloader test########')
random_idx = np.random.randint(0, len(train_data))
print('Tokenised sentence: \n    ', vars(train_data.examples[random_idx])['sentence'])
print('Original sentence: \n    ', tokenizer.convert_ids_to_tokens(vars(train_data.examples[random_idx])['sentence']))
print('Lexical complexity label:\n    ', vars(train_data.examples[random_idx])['label'], '\n')

print('########Iterator test########')
for batch in train_it:
    sentence, label = batch.sentence[0].cpu().tolist(), batch.label[0].cpu().numpy()
    print('Tokenised sentence: \n    ', sentence)
    print('Original sentence: \n    ', tokenizer.convert_ids_to_tokens(sentence))
    print('Lexical complexity label:\n    ', label)
    break

########Dataloader test########
Tokenised sentence: 
     [101, 2057, 16012, 15869, 2135, 7594, 1996, 3815, 1997, 9572, 2094, 1037, 29720, 1999, 1996, 14332, 1997, 2256, 12328, 2077, 1998, 2044, 9099, 6914, 2063, 16341, 2478, 11307, 8132, 4106, 1997, 2522, 28228, 9289, 8153, 2013, 2169, 4111, 1012, 102]
Original sentence: 
     ['[CLS]', 'we', 'bio', '##chemical', '##ly', 'measured', 'the', 'amount', 'of', 'aggregate', '##d', 'a', '##β', 'in', 'the', 'brains', 'of', 'our', 'mice', 'before', 'and', 'after', 'trans', '##gen', '##e', 'suppression', 'using', 'filter', 'trap', 'analysis', 'of', 'co', '##rti', '##cal', 'tissue', 'from', 'each', 'animal', '.', '[SEP]']
Lexical complexity label:
     0.1764705882352941 

########Iterator test########
Tokenised sentence: 
     [2, 2, 2942, 5, 4, 1472, 89, 711, 824, 9, 128, 98, 93, 753, 3105, 11, 4456, 573, 1868, 1918, 4400, 1244, 6, 1472, 767, 5, 374, 153, 2487, 9, 4, 216, 4592, 76, 5, 25, 8, 217, 8, 5, 5128, 71, 62, 7, 359, 98, 100, 5, 345, 14

In [ ]:
# define model: BERT + biLSTM + MLP
class BERT_model(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, layers, bidirectional, dropout):
        super().__init__()
        self.bert = bert
        self.bidirectional = bidirectional
        emb_dim = bert.config.to_dict()['hidden_size']
        
        self.lstm = nn.LSTM(emb_dim, 
                            hidden_dim, 
                            layers,
                            bidirectional=bidirectional,
                            batch_first=True,
                            dropout = 0 if layers < 2 else dropout)
        
        if self.bidirectional:
            fc_input_dim = hidden_dim * 2
        else:
            fc_input_dim = hidden_dim
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(fc_input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        with torch.no_grad():
            embeddings = self.bert(x)[0] # [bs, len, emb_dim]
        
        _, hn = self.lstm(embeddings) # [l * d, bs, emb_dim]
        
        if self.bidirectional:
            hn = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim=1)
        
        hn = self.relu(hn)
        hn = self.fc(hn)
        output = self.sigmoid(hn)
        
        return output

In [ ]:
hidden_dim = 128
output_dim = 1
layers = 3
bidirectional = True
dropout = 0.1

model = BERT_model(bert=bert,
                   hidden_dim=hidden_dim,
                   output_dim=output_dim,
                   layers=layers,
                   bidirectional=bidirectional,
                   dropout=dropout)
model.to(device)

# freeze bert params
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False


print(f'The model has {sum(p.numel() for p in model.parameters()):,} total parameters')
print(f'The model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters')

In [ ]:
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterion = criterion.to(device)

In [37]:
# train function
def train(iterator, model, optimiser, criterion):
    
    epoch_loss = 0
    
    model.train()
    for batch in iterator:
        
        optimizer = zero_grad()
        pred = model(batch.sentence)
        loss = criterion(pre, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
    
    return eposs_loss / len(iterator)

In [38]:
# evaluation function
def eval(iterator, model, criterion):
    model.eval()
    pass

In [ ]:
# main loop
n_epochs = 10
start = time.time()

for epoch in tqdm(n_epochs):
    
    epoch_start = time.time()
    train_loss = train(train_it, model, optimiser, criterion)
    # valid_loss ...
    
    epoch_end = time.time()
    